# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [2]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math
from scipy import stats

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [3]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks.head()


,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV


## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [7]:
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'

for symbol in stocks['Ticker'][:5]:
        print(symbol)
        #symbol = 'AAPL'
        api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
        data = requests.get(api_url).json()
        #print(data['symbol'])
        #print(data.status_code)
        print(data)

A
{'symbol': 'A', 'companyName': 'Agilent Technologies Inc.', 'primaryExchange': 'OYHK, G R KXISCTCE.N ECEWONNA', 'calculationPrice': 'previousclose', 'open': None, 'openTime': None, 'openSource': 'ififoacl', 'close': None, 'closeTime': None, 'closeSource': 'lcfiofai', 'high': None, 'highTime': 1636275321570, 'highSource': 'yneedru epeaiim 5 dcl1t', 'low': None, 'lowTime': 1689673153951, 'lowSource': 'netp5liey rei d1acemdu ', 'latestPrice': 122.65, 'latestSource': 'Previous close', 'latestTime': 'December 31, 2020', 'latestUpdate': 1658862739231, 'latestVolume': None, 'iexRealtimePrice': None, 'iexRealtimeSize': None, 'iexLastUpdated': None, 'delayedPrice': None, 'delayedPriceTime': None, 'oddLotDelayedPrice': None, 'oddLotDelayedPriceTime': None, 'extendedPrice': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPriceTime': None, 'previousClose': 118.69, 'previousVolume': 1368671, 'change': 0, 'changePercent': 0, 'volume': None, 'iexMarketPercent': None, 'iexVolum

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

34.91

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [9]:
final_dataframe = pd.DataFrame(columns=my_columns)
#final_dataframe
for symbol_string in symbol_strings:
    batch_api_url= f'https://sandbox.iexapis.com/stable/stock/market/batch/?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    print(batch_api_url)
    data = requests.get(batch_api_url).json()
    for symbol in symbol_string.split(','):
        
        
       final_dataframe=final_dataframe.append(
        pd.Series(
            [
            symbol,
            data[symbol]['quote']['latestPrice'],
            data[symbol]['quote']['peRatio'],
            'N/A'
            ],
            index = my_columns

        ),
            ignore_index = True

        )
  
final_dataframe

https://sandbox.iexapis.com/stable/stock/market/batch/?symbols=A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA&types=quote&token=Tpk_059b97af715d417d9f49f50b51b1c448
https://sandbox.iexapis.com/stable/stock/market/batch/?symbols=CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FT

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,122.28,54.15,N/A
1,AAL,15.85,-1.16,N/A
2,AAP,159.15,23,N/A
3,AAPL,134.55,40.67,N/A
4,ABBV,107.69,24.5,N/A
5,ABC,101.59,13.16,N/A
6,ABMD,329.50,71.9,N/A
7,ABT,114.73,57.71,N/A
8,ACN,263.76,34.36,N/A
9,ADBE,502.97,65.9,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [16]:
final_dataframe.sort_values('Price-to-Earnings Ratio',ascending=True,inplace=True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio']>0][:50]
final_dataframe.reset_index(inplace=True)
final_dataframe

,index,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,340,NRG,38.39,2.33,N/A
1,335,NLOK,20.84,4.17,N/A
2,459,UNM,23.24,4.98,N/A
3,63,BIO,602.25,5.12,N/A
4,19,AIV,5.46,5.27,N/A
5,265,KIM,15.45,6.63,N/A
6,26,ALL,113.95,7.8,N/A
7,152,EBAY,50.85,7.92,N/A
8,304,MET,47.88,8,N/A
9,62,BIIB,251.14,8.39,N/A


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [17]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [18]:
portfolio_input()

Enter the value of your portfolio:1000000


You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [22]:
position_size = float(portfolio_size)/len(final_dataframe.index)
position_size
for row in final_dataframe.index:
    final_dataframe.loc[row,'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[row,'Price'])
final_dataframe

,index,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,340,NRG,38.39,2.33,520
1,335,NLOK,20.84,4.17,959
2,459,UNM,23.24,4.98,860
3,63,BIO,602.25,5.12,33
4,19,AIV,5.46,5.27,3663
5,265,KIM,15.45,6.63,1294
6,26,ALL,113.95,7.8,175
7,152,EBAY,50.85,7.92,393
8,304,MET,47.88,8,417
9,62,BIIB,251.14,8.39,79


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [28]:
rv_columns = [
    'ticker',
    'price',
    'number of shares to buy',
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns=rv_columns)
#rv_dataframe
for symbol_string in symbol_strings:
    batch_api_url= f'https://sandbox.iexapis.com/stable/stock/market/batch/?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    print(batch_api_url)
    data = requests.get(batch_api_url).json()
    for symbol in symbol_string.split(','):
        pe_ratio = data[symbol]['quote']['peRatio']
        ps_ratio = data[symbol]['advanced-stats']['priceToSales']
        try:
            
            ev_ebitda = data[symbol]['advanced-stats']['enterpriseValue']/data[symbol]['advanced-stats']['EBITDA']
            ev_gp = data[symbol]['advanced-stats']['enterpriseValue']/data[symbol]['advanced-stats']['grossProfit']
        except:
            ev_ebitda = np.NaN
            ev_gp = np.NaN
            
        rv_dataframe=rv_dataframe.append(
        pd.Series(
            [
        symbol,
        data[symbol]['quote']['latestPrice'],
        'N/A',
       pe_ratio,
        'N/A',
        ps_ratio,
        'N/A',
        ev_ebitda,
        'N/A',
        ev_gp,
        'N/A',
        'N/A'
            ],
            index = rv_columns

        ),
            ignore_index = True

        )
rv_dataframe

https://sandbox.iexapis.com/stable/stock/market/batch/?symbols=A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA&types=quote,advanced-stats&token=Tpk_059b97af715d417d9f49f50b51b1c448
https://sandbox.iexapis.com/stable/stock/market/batch/?symbols=CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA

,ticker,price,number of shares to buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,119.96,N/A,56.03,N/A,7.12,N/A,33.732710,N/A,13.424847,N/A,N/A
1,AAL,16.45,N/A,-1.17,N/A,0.4029,N/A,-6.263736,N/A,1.398740,N/A,N/A
2,AAP,164.81,N/A,23.39,N/A,1.12,N/A,10.991415,N/A,2.368414,N/A,N/A
3,AAPL,137.87,N/A,40.54,N/A,8.36,N/A,30.311864,N/A,22.214209,N/A,N/A
4,ABBV,109.20,N/A,23.6,N/A,4.85,N/A,15.683104,N/A,9.388669,N/A,N/A
5,ABC,98.42,N/A,13.28,N/A,0.1077,N/A,8.917180,N/A,3.979851,N/A,N/A
6,ABMD,340.10,N/A,73.9,N/A,18.76,N/A,56.461895,N/A,21.558189,N/A,N/A
7,ABT,114.74,N/A,57.96,N/A,6.26,N/A,31.836391,N/A,11.440707,N/A,N/A
8,ACN,273.25,N/A,34.08,N/A,4.1,N/A,19.625369,N/A,11.637450,N/A,N/A
9,ADBE,501.40,N/A,65.87,N/A,19.5,N/A,57.559550,N/A,23.092733,N/A,N/A


Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

,ticker,price,number of shares to buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
40,AON,219.13,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
71,BRK.B,236.44,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,12.00,N/A,10.14,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
136,DISCK,26.98,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
165,ETFC,49.67,N/A,14.44,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
168,EVRG,57.78,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
190,FOX,29.14,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
192,FRC,149.52,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
194,FTI,9.60,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
204,GOOG,1822.34,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [44]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,ticker,price,number of shares to buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [43]:
for column in [ 'Price-to-Earnings Ratio', 'Price-to-Book Ratio', 'EV/EBITDA', 'EV/GP']:
    #print(column)
    rv_dataframe.fillna(rv_dataframe[column].mean(),inplace=True)
rv_dataframe

,ticker,price,number of shares to buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,119.96,N/A,56.03000,N/A,7.12000,N/A,33.732710,N/A,13.424847,N/A,N/A
1,AAL,16.45,N/A,-1.17000,N/A,0.40290,N/A,-6.263736,N/A,1.398740,N/A,N/A
2,AAP,164.81,N/A,23.39000,N/A,1.12000,N/A,10.991415,N/A,2.368414,N/A,N/A
3,AAPL,137.87,N/A,40.54000,N/A,8.36000,N/A,30.311864,N/A,22.214209,N/A,N/A
4,ABBV,109.20,N/A,23.60000,N/A,4.85000,N/A,15.683104,N/A,9.388669,N/A,N/A
5,ABC,98.42,N/A,13.28000,N/A,0.10770,N/A,8.917180,N/A,3.979851,N/A,N/A
6,ABMD,340.10,N/A,73.90000,N/A,18.76000,N/A,56.461895,N/A,21.558189,N/A,N/A
7,ABT,114.74,N/A,57.96000,N/A,6.26000,N/A,31.836391,N/A,11.440707,N/A,N/A
8,ACN,273.25,N/A,34.08000,N/A,4.10000,N/A,19.625369,N/A,11.637450,N/A,N/A
9,ADBE,501.40,N/A,65.87000,N/A,19.50000,N/A,57.559550,N/A,23.092733,N/A,N/A


Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [36]:
rv_dataframe.columns

Index(['ticker', 'price', 'number of shares to buy', 'Price-to-Earnings Ratio',
       'PE Percentile', 'Price-to-Book Ratio', 'PB Percentile', 'EV/EBITDA',
       'EV/EBITDA Percentile', 'EV/GP', 'EV/GP Percentile', 'RV Score'],
      dtype='object')

## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [74]:
metrics = {
    'Price-to-Earnings Ratio':'PE Percentile', 
    'Price-to-Book Ratio': 'PB Percentile', 
    'EV/EBITDA':'EV/EBITDA Percentile', 
    'EV/GP':'EV/GP Percentile'
}

for metric in metrics.keys():
    for row in rv_dataframe.index:
        rv_dataframe.loc[row, metrics[metric]] = stats.percentileofscore(rv_dataframe[metric],rv_dataframe.loc[row, metric])/100
    #print(metric)
rv_dataframe

,ticker,price,number of shares to buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,23.41,2135,5.14,0.54,0.3946,0.22,2.557747,0.16,0.392047,0.04,5.44554
1,AIG,39.09,1279,-6.80,0.38,0.7208,0.44,4.344107,0.24,0.701362,0.12,5.59406
2,F,8.97,5574,-16.77,0.26,0.2720,0.1,4.476723,0.28,1.937291,0.62,5.79208
3,GPS,21.01,2379,-7.21,0.36,0.5433,0.3,-10.285609,0.1,1.552018,0.5,5.79208
4,AAL,16.45,3039,-1.17,0.48,0.4029,0.24,-6.263736,0.12,1.398740,0.42,6.33663
5,HFC,25.91,1929,-27.00,0.2,0.2965,0.12,6.012327,0.4,2.214988,0.66,6.38614
6,KR,33.08,1511,8.60,0.6,0.1883,0.08,5.533856,0.34,1.143132,0.3,6.68317
7,MET,48.68,1027,7.90,0.58,0.6670,0.4,4.436172,0.26,0.628151,0.1,6.93069
8,ALL,112.54,444,7.70,0.56,0.7563,0.48,2.566663,0.18,0.743047,0.14,7.0297
9,HIG,51.33,974,10.55,0.62,0.8553,0.54,4.178971,0.22,0.838628,0.18,8.46535


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [75]:
from statistics import mean

for row in rv_dataframe.index:
    rv_score =[]
    for i in ['PE Percentile','PB Percentile','EV/EBITDA Percentile','EV/GP Percentile']:
        rv_score.append(rv_dataframe.loc[row,i])
    #print(rv_score)
    rv_dataframe.loc[row,'RV Score']= mean(rv_score)
rv_dataframe

        

,ticker,price,number of shares to buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,23.41,2135,5.14,0.54,0.3946,0.22,2.557747,0.16,0.392047,0.04,0.24
1,AIG,39.09,1279,-6.80,0.38,0.7208,0.44,4.344107,0.24,0.701362,0.12,0.295
2,F,8.97,5574,-16.77,0.26,0.2720,0.1,4.476723,0.28,1.937291,0.62,0.315
3,GPS,21.01,2379,-7.21,0.36,0.5433,0.3,-10.285609,0.1,1.552018,0.5,0.315
4,AAL,16.45,3039,-1.17,0.48,0.4029,0.24,-6.263736,0.12,1.398740,0.42,0.315
5,HFC,25.91,1929,-27.00,0.2,0.2965,0.12,6.012327,0.4,2.214988,0.66,0.345
6,KR,33.08,1511,8.60,0.6,0.1883,0.08,5.533856,0.34,1.143132,0.3,0.33
7,MET,48.68,1027,7.90,0.58,0.6670,0.4,4.436172,0.26,0.628151,0.1,0.335
8,ALL,112.54,444,7.70,0.56,0.7563,0.48,2.566663,0.18,0.743047,0.14,0.34
9,HIG,51.33,974,10.55,0.62,0.8553,0.54,4.178971,0.22,0.838628,0.18,0.39


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [76]:
rv_dataframe.sort_values('RV Score',ascending=True,inplace=True)
rv_dataframe = rv_dataframe[:50]
#len(rv_dataframe)
rv_dataframe.reset_index(drop=True,inplace=True)
rv_dataframe

,ticker,price,number of shares to buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,23.41,2135,5.14,0.54,0.3946,0.22,2.557747,0.16,0.392047,0.04,0.24
1,AIG,39.09,1279,-6.80,0.38,0.7208,0.44,4.344107,0.24,0.701362,0.12,0.295
2,BA,214.89,232,-27.59,0.18,2.0800,0.96,-28.671346,0.04,-202.519029,0.02,0.3
3,F,8.97,5574,-16.77,0.26,0.2720,0.1,4.476723,0.28,1.937291,0.62,0.315
4,GPS,21.01,2379,-7.21,0.36,0.5433,0.3,-10.285609,0.1,1.552018,0.5,0.315
5,AAL,16.45,3039,-1.17,0.48,0.4029,0.24,-6.263736,0.12,1.398740,0.42,0.315
6,KR,33.08,1511,8.60,0.6,0.1883,0.08,5.533856,0.34,1.143132,0.3,0.33
7,MET,48.68,1027,7.90,0.58,0.6670,0.4,4.436172,0.26,0.628151,0.1,0.335
8,ALL,112.54,444,7.70,0.56,0.7563,0.48,2.566663,0.18,0.743047,0.14,0.34
9,PRU,81.08,616,-192.57,0.04,0.5246,0.26,17.670780,1,0.503704,0.06,0.34


## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [77]:
portfolio_input()

Enter the value of your portfolio:2500000


In [78]:
position_size = float(portfolio_size)/len(rv_dataframe.index)
position_size

for row in rv_dataframe.index:
    rv_dataframe.loc[row,'number of shares to buy']= math.floor(position_size/rv_dataframe.loc[row,'price'])

#rv_dataframe.drop('Number of Shares to Buy',axis=1,inplace=True)
rv_dataframe

,ticker,price,number of shares to buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,23.41,2135,5.14,0.54,0.3946,0.22,2.557747,0.16,0.392047,0.04,0.24
1,AIG,39.09,1279,-6.80,0.38,0.7208,0.44,4.344107,0.24,0.701362,0.12,0.295
2,BA,214.89,232,-27.59,0.18,2.0800,0.96,-28.671346,0.04,-202.519029,0.02,0.3
3,F,8.97,5574,-16.77,0.26,0.2720,0.1,4.476723,0.28,1.937291,0.62,0.315
4,GPS,21.01,2379,-7.21,0.36,0.5433,0.3,-10.285609,0.1,1.552018,0.5,0.315
5,AAL,16.45,3039,-1.17,0.48,0.4029,0.24,-6.263736,0.12,1.398740,0.42,0.315
6,KR,33.08,1511,8.60,0.6,0.1883,0.08,5.533856,0.34,1.143132,0.3,0.33
7,MET,48.68,1027,7.90,0.58,0.6670,0.4,4.436172,0.26,0.628151,0.1,0.335
8,ALL,112.54,444,7.70,0.56,0.7563,0.48,2.566663,0.18,0.743047,0.14,0.34
9,PRU,81.08,616,-192.57,0.04,0.5246,0.26,17.670780,1,0.503704,0.06,0.34


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [86]:
writer = pd.ExcelWriter('value strategy.xlsx',engine='xlsxwriter')
rv_dataframe.to_excel(writer,sheet_name='value strategy',index=False)

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [87]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0.0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [88]:
column_format = {
    'A':['ticker',string_template],
    'B':['price',dollar_template],
   'C' :['number of shares to buy',integer_template],
   'D': ['Price-to-Earnings Ratio',float_template],
    'E':['PE Percentile',percent_template],
   'F' :['Price-to-Book Ratio',float_template],
    'G':['PB Percentile',percent_template],
   'H' :['EV/EBITDA',float_template],
    'I':['EV/EBITDA Percentile',percent_template],
    'J':['EV/GP',float_template],
    'K':['EV/GP Percentile',percent_template],
    'L':['RV Score',percent_template],

    
}
    
for column in column_format.keys():
    writer.sheets['value strategy'].set_column(f'{column}:{column}',22,column_format[column][1])

writer.save()


## Saving Our Excel Output
As before, saving our Excel output is very easy: